<a href="https://colab.research.google.com/github/pckuo/dale-regression/blob/main/playground_pck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import grad, random, lax
import seaborn as sns

sns.set(font_scale=1.3)
sns.set_style("whitegrid")
sns.set_palette("rocket", n_colors=10)

### Train a Linear Perceptron with Gradient Descent